In [1]:
import pickle
import pandas as pd
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import re
import time
import itertools

In [2]:
sg = pickle.load(open('sg_post_df.pkl', 'br'))
sg = sg[['post_number', 'post_txt', 'groups']]
sg = sg.drop_duplicates()

In [3]:
sg.sample(5)


,post_number,post_txt,groups
3441,602174,"['n ', 'Hi, Iu0027m new here. Not sure how th...",[' HIV']
11427,555843,"['n ', 'Hi, Iu0027m new to the group. I need...",[' Sleep Apnea']
36108,150825,['n Hello everyone! I am new on here. I am 27...,[' Obesity']
9575,538232,['n I do know what to do I have been throwing...,[' Trying To Conceive']
1082,590795,"['n ', 'Is anyone a T1 diabetic as everyone I...",[' Diabetes']


In [4]:
sg['post_number'] = sg['post_number'].astype(int)
sg['groups'] = sg['groups'].str.strip('[]\'').str.split(',')
sg['post_txt'] = sg['post_txt'].str.strip('[]\'')
sg.sample(5)

,post_number,post_txt,groups
18111,403378,"n ', 'Is anyone here being bullied that can g...",[ Bullying]
4519,523422,"n ', 'I have a cousin in the navy and I miss ...",[ Military Family]
4343,596828,"n ', 'https://www.bbc.co.uk/programmes/m00019...",[ Personality Disorder]
22680,368907,"n (coinued from above). Anyhow, now my physi...",[ Incest Survivors]
15029,383304,n Well Iu0027m down to 3.00 dollers in loose ...,[ Shopping Addiction]


In [6]:
pickle.dump(sg, open('sg_post_dfv2.pkl', 'bw'))